# **Figure 1**. (a–c) Average clustering coefficient $\langle C \rangle$ and temporal evolution of (d–f) global clustering coefficient $C$, (g–i) small-world index $\omega$, and (j–l) average shortest path length $L$.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# === Style settings ===
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

fontsize_labels = 30
fontsize_ticks = 25
fontsize_titles = 35
fontsize_panel_labels = 30
fontsize_legend = 25
panel_spacing = 0.25
point_size_scatter = 150

# === Color map ===
color_map = {
    "0.100": 'tab:orange', "0.150": 'tab:cyan', "0.200": 'tab:purple', "0.250": 'tab:brown',
    "0.300": 'b', "0.350": 'tab:pink', "0.360": 'tab:olive', "0.370": 'tab:gray',
    "0.400": 'g', "0.450": 'tab:blue', "0.500": 'r'
}

# === Mapping from p to <k> ===
p_to_k = {
    "0.060": 18,
    "0.120": 36,
    "0.180": 54
}

# === Metrics and axis limits ===
dynamic_metrics = ["clustering", "omega", "path_length"]
metric_labels = [r"$\langle C \rangle$", r"$C$", r"$\omega$", r"$L$"]
y_limits = [
    (0.0, 0.9),   # <C>
    (0.0, 0.9),   # C
    (0.0, 10.0),  # omega
    (1.7, 3.2)    # L
]

# === Initialize figure ===
fig, axs = plt.subplots(3, 4, figsize=(30, 20))
panel_labels = [f"({chr(97+i)})" for i in range(12)]
panel_label_matrix = np.array(panel_labels).reshape(4, 3).T

# === Get epsilon values and legend once from the first file
first_folder = next(iter(p_to_k))
df_temp = pd.read_csv(os.path.join(f"results/results_p{first_folder}", "network_metrics.txt"))
epsilons_for_legend = sorted(df_temp["epsilon"].unique())
legend_handles = [
    plt.Line2D([0], [0], color=color_map[f"{eps:.3f}"], lw=2, label=rf"$\varepsilon={eps:.2f}$")
    for eps in epsilons_for_legend
]

# === Loop through each p folder ===
for row_idx, (p_str, k_val) in enumerate(p_to_k.items()):
    folder = f"results/results_p{p_str}"
    path = os.path.join(folder, "network_metrics.txt")
    if not os.path.exists(path):
        continue

    df = pd.read_csv(path)
    df.rename(columns={"<s>": "s_mean", "<s>_std": "s_std"}, inplace=True)
    epsilon_values = sorted(df["epsilon"].unique())
    n_last = 100000

    # Panel for <C>
    ax = axs[row_idx, 0]
    ax.set_ylim(y_limits[0])
    avg_clustering = []
    std_clustering = []

    for eps in epsilon_values:
        df_eps = df[df["epsilon"] == eps]
        max_step = df_eps["time_step"].max()
        df_final = df_eps[df_eps["time_step"] >= max_step - n_last]
        grouped = df_final.groupby("simulation")["clustering"].mean()
        avg_clustering.append(grouped.mean())
        std_clustering.append(grouped.std())

    eps_array = np.array(epsilon_values)
    colors = [color_map.get(f"{eps:.3f}", 'k') for eps in eps_array]
    ax.scatter(eps_array, avg_clustering, c=colors, s=point_size_scatter, zorder=2)
    ax.plot(eps_array, avg_clustering, color='black', linewidth=1.2, zorder=1)
    ax.errorbar(eps_array, avg_clustering, yerr=std_clustering,
                fmt='none', ecolor='black', elinewidth=2.0, capsize=3, capthick=1.5, zorder=3)

    if row_idx == 2:
        ax.set_xlabel(r"$\varepsilon$", fontsize=fontsize_labels)
    if row_idx == 0:
        ax.set_title(metric_labels[0], fontsize=fontsize_titles)

    ax.set_ylabel(rf"$\langle k \rangle = {k_val}$", fontsize=fontsize_labels)
    ax.grid(True)
    ax.tick_params(axis='both', labelsize=fontsize_ticks)
    ax.text(0.02, 0.93, panel_label_matrix[row_idx, 0], transform=ax.transAxes,
            fontsize=fontsize_panel_labels, fontweight='bold', va='top', ha='left')

    # Panels for C, omega, L
    for col_offset, (metric, label, y_lim) in enumerate(zip(dynamic_metrics, metric_labels[1:], y_limits[1:]), start=1):
        ax = axs[row_idx, col_offset]
        ax.set_ylim(y_lim)

        for eps in epsilon_values:
            df_eps = df[df["epsilon"] == eps]
            grouped = df_eps.groupby("time_step")[metric].agg(['mean', 'std']).reset_index()
            eps_str = f"{eps:.3f}"
            color = color_map.get(eps_str, 'k')

            ax.plot(grouped["time_step"], grouped["mean"], color=color, label=rf"$\varepsilon={eps:.2f}$")
            ax.fill_between(grouped["time_step"],
                            grouped["mean"] - grouped["std"],
                            grouped["mean"] + grouped["std"],
                            color=color, alpha=0.2)

        if row_idx == 2:
            ax.set_xlabel("Time steps", fontsize=fontsize_labels)
        if row_idx == 0:
            ax.set_title(label, fontsize=fontsize_titles)

        ax.grid()
        ax.tick_params(axis='both', labelsize=fontsize_ticks)
        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
        ax.xaxis.get_offset_text().set_fontsize(fontsize_ticks)

        ax.text(0.02, 0.93, panel_label_matrix[row_idx, col_offset], transform=ax.transAxes,
                fontsize=fontsize_panel_labels, fontweight='bold', va='top', ha='left')

# === Global legend (bottom center)
fig.legend(handles=legend_handles,
           loc='upper center',
           bbox_to_anchor=(0.5, -0.01),
           ncol=len(legend_handles),
           fontsize=fontsize_legend,
           frameon=True,
           handlelength=2.0).get_frame().set_edgecolor('black')

# === Save output
output_dir = "plots_paper/fig1"
os.makedirs(output_dir, exist_ok=True)
plt.tight_layout(rect=[0, 0.06, 1, 1])
fig.subplots_adjust(bottom=0.08, wspace=panel_spacing, hspace=panel_spacing)
plt.savefig(os.path.join(output_dir, "fig1.pdf"), bbox_inches="tight")
plt.savefig(os.path.join(output_dir, "fig1.png"), dpi=300, bbox_inches="tight")
plt.close()

print("✅ Final figure saved as plots_paper/fig1/fig1.[pdf|png]")


✅ Final figure saved as plots_paper/fig1/fig1.[pdf|png]


# **Figure 2 and 3**. Community Structure by Louvain Algorithm.

In [4]:
import os
from pathlib import Path
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import community.community_louvain as community_louvain
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.gridspec import GridSpec


# === k -> p mapping ===
k_to_p = {18: 0.060, 36: 0.120, 54: 0.180}

# === robust loading helpers ===
def _load_txt_flexible(path: Path):
    try:
        return np.loadtxt(path, delimiter=",")  # CSV
    except Exception:
        return np.loadtxt(path)                 # whitespace

def _load_matrix(path: Path):
    if not path.exists(): return None
    A = _load_txt_flexible(path)
    if A.ndim == 1:
        n = int(np.sqrt(A.size))
        if n*n == A.size:
            A = A.reshape(n, n)
    return A

def _load_state(path: Path|None):
    if path is None or not path.exists(): return None
    x = np.load(path) if path.suffix == ".npy" else _load_txt_flexible(path)
    x = np.asarray(x)
    if x.ndim == 2 and 1 in x.shape: x = x.reshape(-1)
    return x

def _find_state_file(root_p: Path, sim_id: int, eps_str: str, step: int):
    """
    Searches for states in both possible locations:
      A) results_p*/simulation_X/states/eps_XXX/
      B) results_p*/states/simulation_X/eps_XXX/
    and with a flexible name containing eps and step.
    """
    candidates = [
        root_p / f"simulation_{sim_id}" / "states" / f"eps_{eps_str}",
        root_p / "states" / f"simulation_{sim_id}" / f"eps_{eps_str}",
    ]
    eps_pat  = re.compile(r"eps[_-]?" + re.escape(eps_str) + r"0*", re.I)
    step_pat = re.compile(rf"step[_-]?{step}$", re.I)

    for base in candidates:
        if not base.exists(): 
            continue
        
        exact = base / f"states_eps_{eps_str}_step_{step}.txt"
        if exact.exists(): return exact
        
        for f in base.iterdir():
            if f.is_file() and f.suffix in {".txt", ".npy"}:
                name = f.name.lower()
                if "step" in name and eps_pat.search(name) and step_pat.search(name):
                    return f
    return None

def _louvain_order(A: np.ndarray, x: np.ndarray, tie_break="none"):
    G = nx.from_numpy_array(A)
    part = community_louvain.best_partition(G, weight='weight')
    Q = community_louvain.modularity(part, G, weight='weight')

    if tie_break == "state":                # current (smooth gradient)
        order = sorted(part, key=lambda i: (part[i], x[i]))
    elif tie_break == "id":                 # within community by index
        order = sorted(part, key=lambda i: (part[i], i))
    elif tie_break == "random":             # within community random
        rng = np.random.default_rng(0)      # fix seed if you want reproducibility
        order = sorted(part, key=lambda i: (part[i], rng.random()))
    else:                                   # "none": same as in the paper
        order = sorted(part, key=lambda i: part[i])

    return np.array(order, dtype=int), Q

def plot_louvain_grid_four(
    k: int,
    epsilon: float,
    simulation_id: int,
    steps=(0, 10000, 100000, 300000),
    root_dir="results",
    out_dir="plots_paper/matrices_ordered_by_louvain",
    state_norm="fixed"  # "fixed" uses [-1,1]; "data" uses min/max per panel
    ):
    """
    Requires:
      - k_to_p: dict {k: p}
      - helpers: _find_state_file(base_p, sim_id, eps_str, step) -> Path|None
                 _load_matrix(path) -> np.ndarray|None
                 _load_state(path)  -> np.ndarray|None
                 _louvain_order(A, x) -> (order_idx, modularityQ)
      - imports: matplotlib.pyplot as plt, seaborn as sns, matplotlib.cm as cm,
                 from pathlib import Path
                 from mpl_toolkits.axes_grid1.inset_locator import inset_axes
                 from matplotlib.gridspec import GridSpec
    """
    if k not in k_to_p:
        raise ValueError(f"k={k} no está en {list(k_to_p)}")

    p_str   = f"{k_to_p[k]:.3f}"
    eps_str = f"{epsilon:.3f}"

    base_p = Path(root_dir) / f"results_p{p_str}"
    out_p  = Path(out_dir)
    out_p.mkdir(parents=True, exist_ok=True)

    # ---- figure with one column reserved for the colorbar ----
    fig = plt.figure(figsize=(16, 4), constrained_layout=False)
    gs = GridSpec(1, 5, figure=fig, width_ratios=[1, 1, 1, 1, 0.045])
    axs = [fig.add_subplot(gs[0, i]) for i in range(4)]
    cax = fig.add_subplot(gs[0, 4])  

    cmap_states = cm.autumn
    any_states_present = False

    for ax, step in zip(axs, steps):
        # paths
        M_path = base_p / f"simulation_{simulation_id}" / "networks" / f"eps_{eps_str}" / f"matrix_eps_{eps_str}_step_{step}.txt"
        S_path = _find_state_file(base_p, simulation_id, eps_str, step)

        A = _load_matrix(M_path)
        x = _load_state(S_path)

        if A is None:
            ax.axis("off")
            ax.text(0.5, 0.5, f"Without matrix\nstep={step}", ha="center", va="center", fontsize=12)
            continue

        # sort by Louvain if there are consistent states
        if x is not None and x.size == A.shape[0]:
            order, Q = _louvain_order(A, x)
            A_ord = A[order][:, order]
            x_ord = x[order]
            any_states_present = True
        else:
            Q = None
            A_ord = A
            x_ord = None

        # matrix
        sns.heatmap(A_ord, cmap="Blues", xticklabels=False, yticklabels=False, cbar=False, ax=ax)
        #title = f"Time Step = {step}"
        #title = rf"Time Step = ${step:.0e}$"

        def sci_tex(n: int) -> str:
            if n == 0:
                return r"$0$"
            s = f"{n:.0e}"          # p.ej. "1e+05"
            a, b = s.split('e')
            mant = int(float(a))    # 1
            exp  = int(b)           # 5
            return rf"${mant} \times 10^{{{exp}}}$"

        title = f"Time Step = {sci_tex(step)}"
        ax.set_title(title, fontsize=14)

        if Q is not None:
            title += f"  |  Q = {Q:.3f}"

        #ax.set_title(title, fontsize=14, pad=10)
        ax.set_title(title, fontsize=14, pad=32)

        # embedded status bars
        if x_ord is not None:
            if state_norm == "fixed":
                norm = plt.Normalize(vmin=-1, vmax=1)
            else:
                norm = plt.Normalize(vmin=float(np.min(x_ord)), vmax=float(np.max(x_ord)))

            # top bar (aligned)
            ax_top = inset_axes(
                ax, width="100%", height="5%", loc="upper center",
                bbox_to_anchor=(0, 0.08, 1, 1), bbox_transform=ax.transAxes, borderpad=0
            )
            ax_top.imshow(x_ord.reshape(1, -1), cmap=cmap_states, aspect="auto", norm=norm)
            ax_top.set_xticks([]); ax_top.set_yticks([]); ax_top.set_frame_on(False)

            # right bar (slightly retracted so as not to touch the edge)
            ax_right = inset_axes(
                ax, width="7.0%", height="100%", loc="center right",
                bbox_to_anchor=(0.1, 0, 1, 1), bbox_transform=ax.transAxes, borderpad=0
            )
            ax_right.imshow(x_ord.reshape(-1, 1), cmap=cmap_states, aspect="auto", norm=norm)
            ax_right.set_xticks([]); ax_right.set_yticks([]); ax_right.set_frame_on(False)

    # global colorbar if there was at least one panel with states
    if any_states_present:
        sm = cm.ScalarMappable(cmap=cmap_states, norm=plt.Normalize(vmin=-1, vmax=1))
        sm.set_array([])
        fig.colorbar(sm, cax=cax, label="Node state")

    #fig.suptitle(
    #    f"Louvain matrices — k={k} (p={p_str}), ε={eps_str}, sim={simulation_id}",
    #    fontsize=20, weight="bold", y=0.98
    #)
    plt.subplots_adjust(top=0.88, right=0.92, wspace=0.22) 
    outfile = out_p / f"LouvainGrid_k{k}_p{p_str}_eps_{eps_str}_sim_{simulation_id}.png"
    plt.savefig(outfile, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"✅ Saved: {outfile}")

In [5]:
# Latex format
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

sim_list = [1]
k_list = [18, 36, 54]
eps_list = [0.3, 0.5]

for sim in sim_list:
    for k in k_list:
        for eps in eps_list:
            plot_louvain_grid_four(k=k, epsilon=eps, simulation_id=sim)


✅ Saved: plots_paper\matrices_ordered_by_louvain\LouvainGrid_k18_p0.060_eps_0.300_sim_1.png
✅ Saved: plots_paper\matrices_ordered_by_louvain\LouvainGrid_k18_p0.060_eps_0.500_sim_1.png
✅ Saved: plots_paper\matrices_ordered_by_louvain\LouvainGrid_k36_p0.120_eps_0.300_sim_1.png
✅ Saved: plots_paper\matrices_ordered_by_louvain\LouvainGrid_k36_p0.120_eps_0.500_sim_1.png
✅ Saved: plots_paper\matrices_ordered_by_louvain\LouvainGrid_k54_p0.180_eps_0.300_sim_1.png
✅ Saved: plots_paper\matrices_ordered_by_louvain\LouvainGrid_k54_p0.180_eps_0.500_sim_1.png


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import colormaps as mcm  # avoid DeprecationWarning
import community.community_louvain as community_louvain
from pathlib import Path
from itertools import product
from typing import Tuple, List, Optional, Dict
from matplotlib import colormaps as mcm

# ===================== Loading helpers =====================
def _load_array(path: str) -> np.ndarray:
    """Carga txt/csv. Si falla con coma, prueba whitespace."""
    try:
        return np.loadtxt(path, delimiter=",")
    except ValueError:
        return np.loadtxt(path)

# ===================== Path resolution =====================
def build_paths_generic(
    ROOT: Path,
    sim: int,
    eps: float,
    time_step: int,
    p: Optional[float] = None,
    k: Optional[int] = None,
    N: int = 300,
    require_states: bool = False,
) -> Tuple[str, str, Path]:
    """
    Returns (adj_file, state_file, base_dir) for a (sim, eps, step).
    - If p is given: uses results_p{p}.
    - If k is given: deduces p = k/(N-1) rounded to 3 decimals.
    - If neither p nor k are given: autodetect results_p* via glob.
    """
    eps_tag = f"{eps:.3f}"

    if p is not None:
        base = ROOT / f"results_p{p:.3f}" / f"simulation_{sim}"
    elif k is not None:
        p_calc = round(k / (N - 1), 3)
        base = ROOT / f"results_p{p_calc:.3f}" / f"simulation_{sim}"
    else:
        pattern = f"results_p*/simulation_{sim}/networks/eps_{eps_tag}/matrix_eps_{eps_tag}_step_{time_step}.txt"
        hits = list(ROOT.glob(pattern))
        if not hits:
            raise FileNotFoundError(f"No files found with pattern: {ROOT}\\{pattern}")
        # .../results_pXXX/simulation_sim/networks/eps_xxx/matrix_...
        base = hits[0].parents[2]  # -> .../results_pXXX/simulation_sim

    adj_file = base / "networks" / f"eps_{eps_tag}" / f"matrix_eps_{eps_tag}_step_{time_step}.txt"
    state_file = base / "states"  / f"eps_{eps_tag}" / f"states_eps_{eps_tag}_step_{time_step}.txt"

    if not adj_file.exists():
        raise FileNotFoundError(f"Falta matriz: {adj_file}")
    if require_states and not state_file.exists():
        raise FileNotFoundError(f"Faltan estados: {state_file}")

    return str(adj_file), str(state_file), base


# Optional override k->p (adjust according to your actual folder naming)
k_to_p_override: Dict[int, float] = {18: 0.060, 54: 0.180}

def build_paths_for_combo(
    ROOT: Path, sim: int, eps: float, step: int, k: Optional[int] = None
) -> Tuple[str, str, Path]:
    """
    Uses override p if it exists for that k; otherwise autodetect via glob.
    """
    p_override = None
    if k is not None:
        p_override = k_to_p_override.get(k)
    if p_override is not None:
        return build_paths_generic(ROOT, sim, eps, step, p=p_override, require_states=False)
    return build_paths_generic(ROOT, sim, eps, step, require_states=False)

# ===================== Louvain utils =====================
def _relabel_partition(part: dict[int, int]) -> Tuple[dict[int, int], int]:
    """Relabels communities to 0..K-1 by size (desc) for stable colors."""
    from collections import defaultdict
    groups = defaultdict(list)
    for n, c in part.items():
        groups[c].append(n)
    ordered = sorted(groups.items(), key=lambda kv: (-len(kv[1]), kv[0]))
    remap = {old: new for new, (old, _) in enumerate(ordered)}
    rel = {n: remap[c] for n, c in part.items()}
    return rel, len(ordered)

# ===================== Plot 1×4 =====================
def plot_louvain_graphs_row_from_paths(
    adj_files: List[str],
    state_files: List[str],
    steps: List[int],
    out_file: Optional[str] = None,
    layout_seed: int = 42,
    cmap_name: str = 'autumn',
    node_size: int = 50,
    consistent_layout: bool = False,  # False=layout por-step (robusto). True=layout único (comparabilidad).
) -> None:
    """
    Generates a 1x4 Louvain:
    - Colors by community (discrete palette).
    - Relabeling to 0..K-1 for visual consistency.
    - By default, per-step layout (avoids KeyError if N changes between steps).
      If you want strict comparability, set consistent_layout=True (uses "superset layout").
    """
    assert len(adj_files) == len(state_files) == len(steps) == 4, "Se esperan 4 time-steps."

    graphs, parts, Qs, Ks = [], [], [], []

    for adj, st in zip(adj_files, state_files):
        A = _load_array(adj)
        if Path(st).exists():
            _ = _load_array(st)  # not used for plot; loaded if present
        G = nx.from_numpy_array(A)
        p = community_louvain.best_partition(G)
        p, K = _relabel_partition(p)
        Q = community_louvain.modularity(p, G)
        graphs.append(G); parts.append(p); Qs.append(Q); Ks.append(K)

    # Discrete Colormap
    Kmax = max(Ks) if Ks else 1
    cmap = mcm.get_cmap(cmap_name).resampled(max(Kmax, 1))

    # Layout
    if consistent_layout:
        # Unique layout over a superset of nodes (avoids KeyError if N differs)
        i_best = int(np.argmax(Qs))
        N_max = max(G.number_of_nodes() for G in graphs)
        H = graphs[i_best].copy()
        H.add_nodes_from(range(N_max))  # add missing items as isolates
        pos_global = nx.spring_layout(H, seed=layout_seed)
    else:
        pos_global = None  # will be calculated per-step

    # Plot
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for ax, G, p, Q, step in zip(axes, graphs, parts, Qs, steps):
        pos = pos_global if pos_global is not None else nx.spring_layout(G, seed=layout_seed)
        node_colors = [p[n] for n in G.nodes()]
        nx.draw_networkx_edges(G, pos, alpha=0.15, width=0.4, ax=ax)
        nx.draw_networkx_nodes(
            G, pos, node_size=node_size,
            node_color=node_colors, cmap=cmap,
            vmin=0, vmax=(Kmax - 1 if Kmax > 0 else 0), ax=ax
        )
        ax.axis('off')

        def sci_tex(n: int) -> str:
            """Returns n in LaTeX-friendly scientific notation (e.g., 300000 -> $3 \\times 10^{5}$)."""
            if n == 0:
                return r"$0$"
            sgn = "-" if n < 0 else ""
            n = abs(int(n))
            exp = 0
            while n % 10 == 0:
                n //= 10
                exp += 1
            return rf"${sgn}{n} \times 10^{{{exp}}}$"


        ax.set_title(fr"Time Step = {sci_tex(step)} — $Q={Q:.3f}$", fontsize=11, pad=6)

    plt.tight_layout()
    if out_file:
        Path(out_file).parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(out_file, bbox_inches='tight', dpi=300)
        plt.close()
        print(f"📁 Saved in: {out_file}")
    else:
        plt.show()

# ===================== (Opcional) versión single =====================
def plot_louvain_graph(adj_file: str, state_file: str, out_file: Optional[str] = None, layout_seed: int = 42):
    A = _load_array(adj_file)
    if Path(state_file).exists():
        _ = _load_array(state_file)
    G = nx.from_numpy_array(A)
    partition = community_louvain.best_partition(G)
    modularity = community_louvain.modularity(partition, G)
    print(f"✅ Modularity: {modularity:.6f}")
    pos = nx.spring_layout(G, seed=layout_seed)
    cmap = mcm.get_cmap('autumn').resampled(max(partition.values()) + 1)
    node_colors = [partition[n] for n in G.nodes()]
    plt.figure(figsize=(10, 8))
    nx.draw_networkx_nodes(G, pos, node_size=50, cmap=cmap, node_color=node_colors)
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.axis('off')
    plt.title(f"Louvain — Modularity: {modularity:.3f}", fontsize=14, weight='bold', pad=20)
    if out_file:
        Path(out_file).parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(out_file, bbox_inches='tight', dpi=300)
        plt.close()
        print(f"📁 Saved in: {out_file}")
    else:
        plt.show()

# ===================== DRIVER =====================
if __name__ == "__main__":
    ROOT = Path("results")

    sim_list = [1]
    k_list   = [18, 36, 54]
    eps_list = [0.3, 0.5]
    steps    = [0, 10_000, 100_000, 300_000]

    out_dir = Path("plots_paper/graphs"); out_dir.mkdir(parents=True, exist_ok=True)

    for sim, k, eps in product(sim_list, k_list, eps_list):
        adj_files, state_files = [], []
        ok = True
        for s in steps:
            try:
                # Use override p if it exists for that k; otherwise, autodetect by glob.
                adj_file, state_file, base = build_paths_for_combo(ROOT, sim, eps, s, k=k)
                adj_files.append(adj_file)
                state_files.append(state_file)
            except FileNotFoundError as e:
                print(f"⚠️ falta para sim={sim}, k={k}, eps={eps:.3f}, step={s}: {e}")
                ok = False
                break
        if not ok:
            continue

        eps_tag = str(eps).replace('.', '')
        out_name = f"louvain_row_sim{sim}_k{k}_eps{eps_tag}.png"
        plot_louvain_graphs_row_from_paths(
            adj_files=adj_files,
            state_files=state_files,
            steps=steps,
            out_file=str(out_dir / out_name),
            layout_seed=42,
            cmap_name='autumn',
            node_size=50,
            consistent_layout=False,  # change it to True if you want a single 1x4 layout
        )
        print(f"✔️ sim={sim}, k={k}, eps={eps} done.")


📁 Saved in: plots_paper\graphs\louvain_row_sim1_k18_eps03.png
✔️ sim=1, k=18, eps=0.3 done.
📁 Saved in: plots_paper\graphs\louvain_row_sim1_k18_eps05.png
✔️ sim=1, k=18, eps=0.5 done.
📁 Saved in: plots_paper\graphs\louvain_row_sim1_k36_eps03.png
✔️ sim=1, k=36, eps=0.3 done.
📁 Saved in: plots_paper\graphs\louvain_row_sim1_k36_eps05.png
✔️ sim=1, k=36, eps=0.5 done.
📁 Saved in: plots_paper\graphs\louvain_row_sim1_k54_eps03.png
✔️ sim=1, k=54, eps=0.3 done.
📁 Saved in: plots_paper\graphs\louvain_row_sim1_k54_eps05.png
✔️ sim=1, k=54, eps=0.5 done.


# **Figure 4**. Degree Distribution.

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# === Plot style settings ===
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# === General configuration ===
k_to_p = {18: 0.060, 36: 0.120, 54: 0.180}
epsilons = [0.3, 0.5]
steps = [0, 10_000, 100_000, 300_000]
step_titles = [
    r"Time step $0$",
    r"Time step $1 \times 10^{4}$",
    r"Time step $1 \times 10^{5}$",
    r"Time step $3 \times 10^{5}$"
]

# === Customizable parameters ===
N = 300  # total number of nodes (constant)
max_grado = N - 1
ylim_y = 0.12  # Expected maximum height for P(k)
xlim_x = 180
mostrar_dispersion = False  # Set to True to show ±std shaded areas

# === Visual style ===
bin_edges = np.arange(0, max_grado + 2)  # Integer bins
bin_centers = bin_edges[:-1]
color_map = {0.3: 'blue', 0.5: 'red'}
label_map = {0.3: r"$\varepsilon = 0.3$", 0.5: r"$\varepsilon = 0.5$"}
alfa_mean = 0.6
alfa_std = 0.25

# === File path templates ===
base_template = "results/results_p{:.3f}/simulation_{}/networks/eps_{:.3f}/matrix_eps_{:.3f}_step_{}.txt"
output_dir = "plots_paper/fig4"
os.makedirs(output_dir, exist_ok=True)

# === Loop over different k values ===
for k, p in k_to_p.items():
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    axs = axs.flatten()

    for idx, (step, title) in enumerate(zip(steps, step_titles)):
        ax = axs[idx]

        for eps in epsilons:
            all_probs = []

            for sim_id in range(1, 11):  # Simulations 1 to 10
                path = base_template.format(p, sim_id, eps, eps, step)
                if not os.path.exists(path):
                    print(f"⚠ Not found: {path}")
                    continue

                try:
                    adj = pd.read_csv(path, header=None).values
                    adj = (adj > 0).astype(int)
                    degrees = adj.sum(axis=1)

                    counts, _ = np.histogram(degrees, bins=bin_edges)
                    probs = counts / N  # <- This converts to P(k)
                    all_probs.append(probs)

                except Exception as e:
                    print(f"⚠ Error reading {path}: {e}")
                    continue

            if all_probs:
                probs_array = np.array(all_probs)
                mean_probs = probs_array.mean(axis=0)
                std_probs = probs_array.std(axis=0)

                ax.bar(bin_centers, mean_probs, width=1, alpha=alfa_mean,
                       color=color_map[eps], label=label_map[eps],
                       edgecolor='black', linewidth=0.5)

                if mostrar_dispersion:
                    ax.fill_between(bin_centers,
                                    mean_probs - std_probs,
                                    mean_probs + std_probs,
                                    color=color_map[eps], alpha=alfa_std)

        # === Subplot aesthetics ===
        ax.set_title(title, fontsize=18)
        ax.set_xlabel("Node Degree $k$", fontsize=18)
        ax.set_ylabel(r"$P(k)$", fontsize=18)  # <- Updated label
        ax.tick_params(axis='both', labelsize=14)
        ax.grid(True)
        ax.set_axisbelow(True)
        if ylim_y is not None:
            ax.set_ylim(0, ylim_y)
        if xlim_x is not None:
            ax.set_xlim(0, xlim_x)

        # Disable scientific notation on axes
        ax.ticklabel_format(style='plain', axis='y')
        ax.ticklabel_format(style='plain', axis='x')

    # === Global legend ===
    handles = [plt.Line2D([0], [0], color=color_map[eps], lw=5, label=label_map[eps]) for eps in epsilons]
    fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.02),
               ncol=len(epsilons), fontsize=16, frameon=False)

    # === Save figure ===
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    output_path = os.path.join(output_dir, f"degree_prob_k{k}")
    plt.savefig(output_path + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(output_path + ".pdf", dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✅ Saved: {output_path}.pdf/png")



✅ Saved: plots_paper/fig4\degree_prob_k18.pdf/png
✅ Saved: plots_paper/fig4\degree_prob_k36.pdf/png
✅ Saved: plots_paper/fig4\degree_prob_k54.pdf/png


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# === Plot style settings ===
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# === General configuration ===
k_to_p = {54: 0.180}
epsilons = [0.3, 0.5]
steps = [0, 10_000, 100_000, 300_000]
step_titles = [
    r"Time step $0$",
    r"Time step $1 \times 10^{4}$",
    r"Time step $1 \times 10^{5}$",
    r"Time step $3 \times 10^{5}$"
]

# === Customizable parameters ===
N = 300  # total number of nodes (constant)
max_grado = N - 1
ylim_y = 0.07  # Expected maximum height for P(k)
xlim_x = 180
mostrar_dispersion = False  # Set to True to show ±std shaded areas

# === Visual style ===
bin_edges = np.arange(0, max_grado + 2)  # Integer bins
bin_centers = bin_edges[:-1]
color_map = {0.3: 'blue', 0.5: 'red'}
label_map = {0.3: r"$\varepsilon = 0.3$", 0.5: r"$\varepsilon = 0.5$"}
alfa_mean = 0.6
alfa_std = 0.25

# === File path templates ===
base_template = "results/results_p{:.3f}/simulation_{}/networks/eps_{:.3f}/matrix_eps_{:.3f}_step_{}.txt"
output_dir = "plots_paper/fig4"
os.makedirs(output_dir, exist_ok=True)

# === Loop over different k values ===
for k, p in k_to_p.items():
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    axs = axs.flatten()

    for idx, (step, title) in enumerate(zip(steps, step_titles)):
        ax = axs[idx]

        for eps in epsilons:
            all_probs = []

            for sim_id in range(1, 11):  # Simulations 1 to 10
                path = base_template.format(p, sim_id, eps, eps, step)
                if not os.path.exists(path):
                    print(f"⚠ Not found: {path}")
                    continue

                try:
                    adj = pd.read_csv(path, header=None).values
                    adj = (adj > 0).astype(int)
                    degrees = adj.sum(axis=1)

                    counts, _ = np.histogram(degrees, bins=bin_edges)
                    probs = counts / N  # <- This converts to P(k)
                    all_probs.append(probs)

                except Exception as e:
                    print(f"⚠ Error reading {path}: {e}")
                    continue

            if all_probs:
                probs_array = np.array(all_probs)
                mean_probs = probs_array.mean(axis=0)
                std_probs = probs_array.std(axis=0)

                ax.bar(bin_centers, mean_probs, width=1, alpha=alfa_mean,
                       color=color_map[eps], label=label_map[eps],
                       edgecolor='black', linewidth=0.5)

                if mostrar_dispersion:
                    ax.fill_between(bin_centers,
                                    mean_probs - std_probs,
                                    mean_probs + std_probs,
                                    color=color_map[eps], alpha=alfa_std)

        # === Subplot aesthetics ===
        ax.set_title(title, fontsize=18)
        ax.set_xlabel("Node Degree $k$", fontsize=18)
        ax.set_ylabel(r"$P(k)$", fontsize=18)  # <- Updated label
        ax.tick_params(axis='both', labelsize=14)
        ax.grid(True)
        ax.set_axisbelow(True)
        if ylim_y is not None:
            ax.set_ylim(0, ylim_y)
        if xlim_x is not None:
            ax.set_xlim(0, xlim_x)

        # Disable scientific notation on axes
        ax.ticklabel_format(style='plain', axis='y')
        ax.ticklabel_format(style='plain', axis='x')

    # === Global legend ===
    handles = [plt.Line2D([0], [0], color=color_map[eps], lw=5, label=label_map[eps]) for eps in epsilons]
    fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.02),
               ncol=len(epsilons), fontsize=16, frameon=False)

    # === Save figure ===
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    output_path = os.path.join(output_dir, f"fig4")
    plt.savefig(output_path + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(output_path + ".pdf", dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✅ Saved: {output_path}.pdf/png")



✅ Saved: plots_paper/fig4\fig4.pdf/png
